#Linear Regression on Boston Housing Dataset

##Import the required libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt 

import pandas as pd  
import seaborn as sns 

%matplotlib inline

###Boston Housing Dataset

In [2]:
from sklearn.datasets import load_boston
boston_dataset = load_boston()
# import pandas as pd
# import numpy as np


# data_url = "http://lib.stat.cmu.edu/datasets/boston"
# raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
# data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
# boston_dataset = raw_df.values[1::2, 2]

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np


        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the California housing dataset (i.e.

In [3]:
boston = pd.DataFrame(boston_dataset.data, columns=boston_dataset.feature_names)
boston.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33


In [4]:
boston['MEDV'] = boston_dataset.target

##Preparing the data for training the model

In [5]:
X = pd.DataFrame(np.c_[boston['LSTAT'], boston['RM']], columns = ['LSTAT','RM'])
Y = boston['MEDV']

In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.3, random_state=5)
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

(354, 2)
(152, 2)
(354,)
(152,)


##Training and testing the model

In [7]:
class LinearRegression:
    def __init__(self, learning_rate=0.01, iterations=1000):
        # Hyper-parameters
        self.lr = learning_rate
        self.iterations = iterations

        # Parameters and attributes
        self.params = None
        self.coef_ = None
        self.intercept_ = None

        # Training Features and Samples
        self.X = None
        self.y = None
        self.numOfFeatures = None
        self.numOfSamples = None

    @staticmethod
    def normalize_data(X):
        return (X - np.mean(X, 0)) / np.std(X, 0)

    def fit(self, X, y):
        self.numOfSamples = len(y)
        self.X = self.normalize_data(X)
        # Intercept
        self.X = np.hstack((np.ones((self.numOfSamples, 1)), self.X))

        self.numOfFeatures = np.size(X, 1)
        self.y = np.array(y)[:, np.newaxis]
        self.params = np.zeros((self.numOfFeatures + 1, 1))

        for _ in range(self.iterations):
            grad = np.matmul(self.X.T, (np.matmul(self.X, self.params) - self.y)) / self.numOfSamples
            self.params = self.params - (self.lr * grad)

        self.intercept_ = self.params[0]
        self.coef_ = self.params[1:]

    def predict(self, X):
        n_test_samples = np.size(X, 0)
        X = self.normalize_data(X)
        # Intercept
        X = np.hstack((np.ones((n_test_samples, 1)), X))
        return np.matmul(X, self.params)




In [8]:
lin_model = LinearRegression()
lin_model.fit(X_train, Y_train)

In [9]:
lin_model.coef_

array([[-4.68202131],
       [ 3.36272173]])

In [10]:
lin_model.intercept_

array([22.55524091])

##Model evaluation

In [11]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

# model evaluation for training set
y_train_predict = lin_model.predict(X_train)
rmse = (np.sqrt(mean_squared_error(Y_train, y_train_predict)))
r2 = r2_score(Y_train, y_train_predict)

print("The model performance for training set")
print("--------------------------------------")
print('RMSE is {}'.format(rmse))
print('R2 score is {}'.format(r2))
print("\n")

# model evaluation for testing set
y_test_predict = lin_model.predict(X_test)
rmse = (np.sqrt(mean_squared_error(Y_test, y_test_predict)))
r2 = r2_score(Y_test, y_test_predict)

print("The model performance for testing set")
print("--------------------------------------")
print('RMSE is {}'.format(rmse))
print('R2 score is {}'.format(r2))

The model performance for training set
--------------------------------------
RMSE is 5.291276945019069
R2 score is 0.6493164916572722


The model performance for testing set
--------------------------------------
RMSE is 6.035681389097256
R2 score is 0.6168838722619777
